In [369]:
import robotic as ry
import time
with open("logger.txt","w") as f:
    f.write("")
try:
    del C
    del bot
except:
    pass

C = ry.Config()
# C.watchFile("throwing1.g")
C.addFile("throwing_bare.g")
qHome = C.getJointState()
C.view()
#time.sleep(30)
C.getFrame('cargo').unLink()
C.addFrame("initial").setPosition([-0.6,2.5,0.3]).setShape(ry.ST.marker,[.3])
C.addFrame("throw").setPosition([0,1.8,1.2]).setShape(ry.ST.marker,[.3])
C.view()

0

In [370]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from my_utils import rotation_matrix_to_quaternion, velocity_to_rotation_matrix, find_line_intersections_2d

In [371]:
import numpy as np
from scipy.optimize import minimize

# Define constants
g = 9.81  # Gravity, m/s^2

# Initial positions (robot and bin)
initial_pos = C.getFrame("throw").getPosition()
bin_dims = C.getFrame("bin").getPosition()

# Bin and robot parameters
x_bin, y_bin, z_bin = bin_dims[0], bin_dims[1], bin_dims[2]
x0, y0, z0 = initial_pos[0], initial_pos[1], initial_pos[2]

# Objective function: minimize velocity magnitude
def objective(params):
    v0, theta, phi = params
    return v0

# Constraints: ball lands in the bin's X position
def constraint_x(params):
    v0, theta, phi = params
    vx = v0 * np.cos(theta) * np.cos(phi)
    t_land = (-v0 * np.sin(theta) + np.sqrt((v0 * np.sin(theta))**2 + 2 * g * (z0 - z_bin))) / g
    return x0 + vx * t_land - x_bin

# Constraints: ball lands in the bin's Y position
def constraint_y(params):
    v0, theta, phi = params
    vy = v0 * np.cos(theta) * np.sin(phi)
    t_land = (-v0 * np.sin(theta) + np.sqrt((v0 * np.sin(theta))**2 + 2 * g * (z0 - z_bin))) / g
    return y0 + vy * t_land - y_bin

# Constraints: ball lands in the bin's Z position (height)
def constraint_z(params):
    v0, theta, phi = params
    vz = v0 * np.sin(theta)
    t_land = (-vz + np.sqrt(vz**2 + 2 * g * (z0 - z_bin))) / g
    return z0 + vz * t_land - 0.5 * g * t_land**2 - z_bin

# Constraint: Ball does not hit the bin's sides
def constraint_sides_height(params):
    v0, theta, phi = params
    # Time to reach maximum height
    t_up = v0 * np.sin(theta) / g
    # Ball's height at time t_up (maximum height)
    z_max = z0 + (v0 * np.sin(theta))**2 / (2 * g)
    
    # If the bin's sides are higher than the max height, no problem
    if z_max < z_bin:
        return 0  # No issue, the ball won't hit the sides
    else:
        # If the bin's sides are lower, we need to check at different points in time
        t_down = np.sqrt(2 * (z_max - z_bin) / g)
        z_ball_at_t = z0 + v0 * np.sin(theta) * t_down - 0.5 * g * t_down**2
        return z_ball_at_t - z_bin  # Ball height should not exceed bin sides

# Initial guess and bounds
initial_guess = [5, np.pi / 4, np.pi / 4]  # [velocity, theta, phi]
bounds = [(1, 20), (0, np.pi / 2), (0, 2 * np.pi)]  # Bounds for velocity and angles

# Solve the optimization
result = minimize(objective, initial_guess, constraints=[
    {'type': 'eq', 'fun': constraint_x},
    {'type': 'eq', 'fun': constraint_y},
    {'type': 'eq', 'fun': constraint_z},
    {'type': 'ineq', 'fun': constraint_sides_height}  # Ensure ball avoids sides' height
], bounds=bounds)

# Extract results
v0_opt, theta_opt, phi_opt = result.x
print(f"Optimal velocity: {v0_opt:.2f} m/s, Theta: {np.degrees(theta_opt):.2f}°, Phi: {np.degrees(phi_opt):.2f}°")


Optimal velocity: 4.21 m/s, Theta: 0.00°, Phi: 5.71°


In [372]:
print("Velocity:", v0_opt)
print("Release Angle theta (degrees):", theta_opt)
print("Release Angle phi (degrees):", phi_opt)

v_x = v0_opt * np.cos(theta_opt) * np.cos(phi_opt)
v_y = v0_opt * np.cos(theta_opt) * np.sin(phi_opt)
v_z = v0_opt * np.sin(theta_opt)

velocity = [v_x, v_y, v_z]

C.addFrame("release_frame").setPosition(initial_pos).setShape(ry.ST.marker,[.4]).setColor([1,0,0]).setContact(0)


C.view()
#finding the initial point from the release point and the velocity
#find the backward kinematics and simulate it


def find_initial_point_from_release(release_position, release_velocity, robot_base, height_threshold=0.2, distance_threshold=1.53):
    """
    Finds the initial position for the throwing motion based on the release position and velocity.


    Args:
        C: Robot configuration (KOMO simulation object).
        release_position: The release point (3D coordinates).
        release_velocity: The release velocity vector.
        robot_base: Position of the robot base.
        height_threshold: Minimum allowable height for the initial position.
        distance_threshold: Maximum allowable distance from the robot base.


    Returns:
        initial_position: The computed initial position, if found.
    """
    with open("throwing_params.txt","r") as f:
        distance_xy = f.read().strip()
        print(distance_xy)
        try:
            distance_xy = np.float64(distance_xy)
            print("Successfully converted:", distance_xy)
        except ValueError as e:
            print("Conversion failed:", e)
    velxy_ratio = release_velocity[1]/release_velocity[0]
    x = distance_xy/np.sqrt(1+np.square(velxy_ratio))
    base_position = C.getFrame("l_panda_base").getPosition()
    initial_position  = np.array([-x +base_position[0],-np.multiply(velxy_ratio,x) + base_position[1],0.3])
    iteration = 0
    gravity = np.array([0,0,-9.8])
    iteration = 0
    #resetting the velocity and position after calculating initial position
    velocity = np.array(release_velocity)
    time_step = .1
    position = np.array(release_position)
    while position[2] > height_threshold:
        position += velocity * time_step
        velocity += gravity * time_step
        iteration += 1
        frame = C.addFrame(f"trajectory-{iteration}").setShape(ry.ST.marker,[.2]).setColor([0,0,1]).setPosition(position)
        C.view()
        print(f"Frame {iteration} added with position {frame.getPosition()} new vel:{velocity}")
    return initial_position  # No valid initial position found


def pick_last_object_if_valid(release_position, release_velocity, robot_base):
   """
   Simulates the robot to pick the last object if conditions are met.


   Args:
       C: Robot configuration (KOMO simulation object).
       release_position: The release point (3D coordinates).
       release_velocity: The release velocity vector.
       robot_base: Position of the robot base.


   Returns:
       None
   """
   # Find the initial position
   initial_position = find_initial_point_from_release(release_position, release_velocity, robot_base)


   if initial_position is not None:
       print(f"Initial Position Found: {initial_position}")
       # Visualize the initial position
       C.addFrame("initial_position").setPosition(initial_position).setShape(ry.ST.marker, [0.4]).setContact(0).setColor([0,1,0]).setQuaternion(rotation_matrix_to_quaternion(velocity_to_rotation_matrix(release_velocity)))
       C.view()
       return initial_position
       # Simulate the robot to pick the last object
       # Add your grasping or motion planning logic here
   else:
       print("No valid initial position found.")


initial_pos = pick_last_object_if_valid(np.array(initial_pos),np.array(velocity),C.getFrame("l_panda_base").getPosition())


#go to inital position with y axis being scalar to the velocity vector
def goto_init(gripper_name, initial_pos_name):
   komo = ry.KOMO(C, 1, 1, 0, True)
   komo.addObjective([1.], ry.FS.positionDiff, [gripper_name, initial_pos_name], ry.OT.eq, [1e1], [0, 0, 0])
   komo.addObjective([1], ry.FS.scalarProductYZ, [gripper_name, initial_pos_name], ry.OT.eq, [1e1], [-1])
   ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
   print(ret2)
   return komo

Velocity: 4.206308950790458
Release Angle theta (degrees): 6.573653297051174e-17
Release Angle phi (degrees): 0.09966865396114633
0.986154146165801
Successfully converted: 0.986154146165801
Frame 1 added with position [0.41854338 1.84185434 1.2       ] new vel:[ 4.18543384  0.41854339 -0.98      ]
Frame 2 added with position [0.83708677 1.88370868 1.102     ] new vel:[ 4.18543384  0.41854339 -1.96      ]
Frame 3 added with position [1.25563015 1.92556302 0.906     ] new vel:[ 4.18543384  0.41854339 -2.94      ]
Frame 4 added with position [1.67417354 1.96741736 0.612     ] new vel:[ 4.18543384  0.41854339 -3.92      ]
Frame 5 added with position [2.09271692 2.0092717  0.22      ] new vel:[ 4.18543384  0.41854339 -4.9       ]
Frame 6 added with position [ 2.5112603   2.05112603 -0.27      ] new vel:[ 4.18543384  0.41854339 -5.88      ]
Initial Position Found: [-0.98126005  1.90187399  0.3       ]


In [373]:
def pre_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 3, 1, 0, True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) 
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)

    komo.addObjective([1., 3.], ry.FS.positionRel, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0, 0, 0])  

    # komo.addObjective([1., 3.], ry.FS.scalarProductXX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 
    komo.addObjective([1., 3.], ry.FS.scalarProductYX, [gripper_name, grasp_frame_name], ry.OT.eq, [1e2], [1]) 
    # komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [gripper_name, grasp_frame_name], ry.OT.eq, [1e1], [0]) 

    komo.addObjective([1., 3.], ry.FS.scalarProductZZ, [grasp_frame_name, gripper_name], ry.OT.eq, [1e1], [1]) 


    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo
qHome = C.getJointState()
def post_grasp_komo(C, gripper_name, grasp_frame_name, q0, qHome):
    komo = ry.KOMO(C, 1, 1, 0, True)
    komo.addObjective([], ry.FS.positionDiff, ["l_gripper","initial_position"], ry.OT.eq, [1e1], [0,0,0])
    komo.addObjective([], ry.FS.scalarProductYZ, ["l_gripper","initial_position"], ry.OT.eq, [1e1], [-1])
    ret = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    print(ret)
    return komo
q0 = C.getJointState()
komo0 = pre_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path0 = komo0.getPath()

komo = post_grasp_komo(C, 'l_gripper', 'cargo', q0, qHome)
path = komo.getPath()
bot = ry.BotOp(C, useRealRobot=False)
time.sleep(5)
bot.home(C)
# print(path.shape)
bot.move(path0, [1.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

#bot.gripperCloseGrasp(ry._left, 'cargo',speed=1000000000000,width=0.04)
bot.gripperClose(ry._left,width=0.06) 
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)
bot.move(path, [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)
#C.getFrame('cargo').unLink()
C.attach('l_gripper', 'cargo')

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):16838.8 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    11525.8  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    6792.77  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    3614.39  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    2034.15  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    1039.93  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):    416.787  ACCEPT
--newton-- it:   7  |Delta|:        0.2  alpha:          1  evals:   8  f(y):     87.979  ACCEPT
--newton-- it:   8  |Delta|:    0.17913  alpha:          1  evals:   9  f(y):    6.68028  ACCEPT
--newton-- it:   9  |Delta|:  0.0925512  alpha:          1  evals:  10  f(y

In [374]:
# Add frame for the release position
release_frame = C.addFrame('release')
release_frame.setShape(ry.ST.marker, size=[0])
release_frame.setPosition(initial_pos)
C.view()
time.sleep(1)

In [375]:
#code for calculating velocity
def vel_komo():
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 1, True)
    # komo.addObjective([], ry.FS.position, ["l_gripper"], ry.OT.eq, [1e-1], [4.2,-3.5,4],1)
    komo.addObjective([], ry.FS.position, ["l_gripper"], ry.OT.eq, [1e-1], np.array(velocity),1)
    komo.addObjective([],ry.FS.positionDiff,["l_gripper","release_frame"],ry.OT.sos,[1e0],[0,0,0])
    ret2 = ry.NLP_Solver(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=4).solve()
    #komo.addObjective([1.],ry.FS.scalarProductXZ,["base","l_gripper"],ry.OT.eq,[1e1],[-1],0)
    #print(komo.report())
    print(f"ret!!!: {ret2}")
    return komo
komo = vel_komo()
gripper_frame = C.getFrame("l_gripper")
# bot = ry.BotOp(C,False)
# time.sleep(5)
path = komo.getPath()
print(f"path size: {path.size}")
bot.move(path,[1.])
print(f"bot initial end time:{bot.getTimeToEnd()}")
time.sleep(0.60)
bot.sync(C,0.001)
print(f"bot end time:{bot.getTimeToEnd()}")
bot.gripperMove(ry._left,width=1)
C.addFrame("actual_release").setPosition(gripper_frame.getPosition()).setShape(ry.ST.marker,[.2]).setColor([1,1,0])

# while bot.gripperDone(ry._left):
#     bot.sync(C,.1)
print(f"bot end time:{bot.getTimeToEnd()}")
while bot.getTimeToEnd()>0:
    bot.sync(C,.1)


====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:ret!!!: { time: 0.00544, evals: 137, done: 1, feasible: 1, sos: 1.66868, f: 0, ineq: 0, eq: 0.256855 }
path size: 7
bot initial end time:1.0
0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):1.85651 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    1.50255  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    1.15965  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    0.84602  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):   0.577357  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):   0.365922  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):     0.2196  ACCEPT
--newton-- it:   7  |Delta|:    0.17647  alpha:          1  evals:   8  f(y):   0.147453  ACCEPT
--newton-- it:   8  |Delta|:  0

In [376]:
time.sleep(4.5)
del C
del bot

-- bot.cpp:~BotOp:135(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:57(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
